<a href="https://colab.research.google.com/github/Panashe1812/Masters-Project/blob/main/Lime_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json
from lime import lime_image
from skimage.segmentation import mark_boundaries
import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
img = get_image('./val_/Latino_Hispanic/1024.jpg')
plt.imshow(img)

In [ ]:
model = models.vgg16(pretrained=True)
#replace the last linear layer from classifier
last_layer = nn.Linear(4096,7)
model.classifier[6] = last_layer

In [ ]:
#path2 ='./vgg16_50_epochs.pth'
path3 = './vgg16_model_29_epoch.pth'
model = (torch.load(path3,map_location=torch.device('cpu')))


In [ ]:
def get_image(path):
    with open(os.path.abspath(path), 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB') 
        


In [ ]:
class_map = {0:'Latino_Hispanic',
             1:'East Asian',
             2:'Indian',
             3:'Middle Eastern',
             4:'Black',
             5:'Southeast Asian',
             6:'White'
             }

In [ ]:
img = get_image('./val_/Latino_Hispanic/1024.jpg')
img_t = get_input_tensors(img)
logits = model(img_t)

In [ ]:
logits

In [ ]:
probs = torch.softmax(logits, dim=1)
probs5 = probs.topk(5)
tuple((p,c, class_map[c]) for p, c in zip(probs5[0][0].detach().numpy(), probs5[1][0].detach().numpy()))

In [ ]:
model.eval()

In [ ]:
def get_pil_transform(): 
    transf = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224)
    ])    

    return transf

def get_preprocess_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])     
    transf = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])    

    return transf    

pill_transf = get_pil_transform()
preprocess_transform = get_preprocess_transform()

In [ ]:
def batch_predict(images):
    model.eval()
    batch = torch.stack(tuple(preprocess_transform(i) for i in images), dim=0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    batch = batch.to(device)
    
    logits = model(batch)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

In [ ]:
test_image_1241 = batch_predict([pill_transf(img)])
class_map[test_pred.squeeze().argmax()]


In [ ]:
def Lime_explain(test_pred):
    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(np.array(pill_transf(img)), 
                                             batch_predict, # classification function
                                             top_labels=5, 
                                             hide_color=0, 
                                             num_samples=3000) # number of images that will be sent to classification function
    
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
    img_boundry1 = mark_boundaries(temp/255.0, mask)
    plt.imshow(img_boundry1)
    plt.title('Hispanic Class, 1000 samples')

In [ ]:
def apply_mask(explanation)


temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
img_boundry1 = mark_boundaries(temp/255.0, mask)
plt.imshow(img_boundry1)
plt.title('Hispanic Class, 1000 samples')

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
img_boundry2 = mark_boundaries(temp/255.0, mask)
plt.imshow(img_boundry2)

In [ ]:
class_light_skin =[ 3850, 4631, 7061, 5477, 4399, 6435, 7038 ,7276, 8883, 9438, 2402,3691, 3752 ,5911 ,5915 ,3935 ,3962 ,5291 ,5480, 6297, 6483, 6435, 7038 ,7197 ,7250, 7415,8291]

class_dark_skin = [9112, 9487, 9488, 9306,9306, 9765, 10630, 11299, 11948, 2634, 3119, 2634, 3695, 7607, 5171, 7327, 9988, 2658, 11727,10001]
hisp_misclas = [class_light_skin,class_dark_skin]
path =""

for typ in hisp_misclas:
    for idx in typ:
        path = "./train_/Latino_Hispanic/"+ str(idx)+".jpg"
        img =get_image(path)
        saliency(img, model)
        
        

In [ ]:
img1 = get_image('./val_/Latino_Hispanic/1024.jpg')

In [ ]:
test_pred = batch_predict([pill_transf(img)])
class_map[test_pred.squeeze().argmax()]


In [ ]:
# image paths
path = "./Gradcam vis/test images/"
img_1241 = "1241.jpg"
path_1241= path + img_1241

In [ ]:
img_1241= get_image(path_1241)

In [ ]:
test_img_pred = batch_predict([pill_transf(img_1241)])
class_map[test_img_pred.squeeze().argmax()]


In [ ]:
Lime_explain(test_img_pred)